In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/MoneyLion/MoneyLionTakeHome_MLS_MichelleAdeline/advanced/finBERT_transaction_classification/

In [ ]:
%ls

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import os
import logging
import sys
# sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools
from finbert.custom_finbert import CustomFinBert

# project_dir = Path.cwd().parent
project_dir = Path.cwd()
pd.set_option('max_colwidth', -1)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

c:\ProgramData\Anaconda3\envs\finbert\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
c:\ProgramData\Anaconda3\envs\finbert\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and

In [2]:
standard_dir = project_dir.parent.parent

# ! TODO: PUT THE PREPROCESSING CODE HERE AND STORE PROCESSED DF

# Prepare model

In [4]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
final_model_dir = project_dir/'models'/'classification_model'
data_dir = standard_dir/"Data"
# classification_data_path= standard_dir/'Data'/'sentiment_data'

## Configure training parameters

In [5]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

## load the pretrained language model trained on Reuters TRC2
bertmodel = AutoModelForSequenceClassification.from_pretrained(
    lm_path, cache_dir=None, num_labels=30)

Some weights of the model checkpoint at c:\Programming\MoneyLion\standard\advanced\finBERT_transaction_classification\models\language_model\finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

In [6]:
custom_finbert = CustomFinBert(bertmodel, 38, 30)

In [7]:
config = Config(
    data_dir=data_dir,
    model_dir=final_model_dir,
    bert_model=custom_finbert,
    num_train_epochs=5,
    max_seq_length=64,
    train_batch_size=32,
    learning_rate=2e-5,
    output_mode="classification",
    warm_up_proportion=0.2,
    local_rank=-1,
    discriminate=True,
    gradual_unfreeze=True,
    numeric_params=38,
    label_colname="category"
)

In [8]:
finbert = FinBert(config, transaction_classification=True)

In [10]:
y = pd.read_csv(standard_dir/"Data"/"eval.csv")["category"]

In [11]:
finbert.prepare_model(label_list=y.unique().tolist())

07/17/2024 13:10:50 - INFO - finbert.finbert -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False


In [12]:
train_examples, numeric_feats = finbert.get_data("train")

In [13]:
model = finbert.create_the_model()

In [14]:
trained_model = finbert.train(train_examples = train_examples, model = model, train_numeric_feats=numeric_feats)

07/17/2024 13:13:25 - INFO - finbert.utils -   *** Example ***
07/17/2024 13:13:25 - INFO - finbert.utils -   guid: 0
07/17/2024 13:13:25 - INFO - finbert.utils -   tokens: [CLS] ac ##h withdrawal am ##mar ##yse hem ##ant ##ne ##x ##press transfer [SEP]
07/17/2024 13:13:25 - INFO - finbert.utils -   input_ids: 101 9353 2232 10534 2572 7849 23274 19610 4630 2638 2595 20110 4651 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/17/2024 13:13:25 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/17/2024 13:13:25 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/17/2024 13:13:25 - INFO - finbert.utils -   label: Transfer Debit (id = 2)
07/17/2024 13:13:44 - INFO - finbert.finbert -   ***** L

Iteration:   0%|          | 0/5013 [00:00<?, ?it/s]

(Iteration 0) Training loss: 3.2839462757110596


Epoch:   0%|          | 0/5 [00:24<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
trained_model